In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
N

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-jrha2fax
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-jrha2fax
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=d5d33f1b90caf643cf2f122e3e084f3194679bb73eff52b3823cc7ee61e46109
  Stored in directory: /tmp/pip-ephem-wheel-cache-iq_kuxkq/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [2]:
%%cu 
#include<iostream>
#include<math.h>

using namespace std;

__global__ void minimum(int *arr) {
    int step = 1;
    int threads = blockDim.x;
    
    while (threads > 0)
    {
        if (threadIdx.x < threads){
            int prev=threadIdx.x * step * 2;
            int next=prev+step;
             if(arr[ prev ] > arr[next])
                arr[ prev ] = arr[ next];
        }

        step *= 2; 
        threads /= 2;
    }
}

__global__ void maximum(int *arr) {
    int step = 1;
    int threads = blockDim.x;
    
    while (threads > 0)
    {
        if (threadIdx.x < threads){
            int prev=threadIdx.x * step * 2;
            int next=prev+step;
             if(arr[ prev ] < arr[next])
                arr[ prev ] = arr[ next];
        }
           

        step *= 2; 
        threads /= 2;
    }
}

__global__ void sum(int *arr)
{
    int step=1;
    int threads=blockDim.x;
    while (threads  > 0)
    {
        if (threadIdx.x < threads){
             arr[(threadIdx.x * step * 2)] += arr[(threadIdx.x * step * 2) + step];

        }
     step*=2;
     threads/=2;
       
    }

}

__global__ void sumOfSquares(int *arr)
{
     arr[threadIdx.x] *= arr[threadIdx.x];
 
    

    
}



int main() {
    
    int n=8;
    int size = n*sizeof(int); 
    
    int *source;
    int *destination, result;
   
    source = new int[n];
     for(int i=0; i<n; i++)  {
        source[i] = rand()%10 +1;
        cout<<source[i]<<"  ";   
    }
    cout<<endl;
    cudaMalloc(&destination, size);

     //SUM
     cudaMemcpy(destination, source, size, cudaMemcpyHostToDevice);
    sum<<<1,n/2>>>(destination);
    cudaMemcpy(&result, destination, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"The sum  is "<<result<<endl;

    //AVG
    float avg=(result*1.0)/(n*1.0);
    cout<<"The average  is "<<avg<<endl;
    
    //MIN
    cudaMemcpy(destination, source, size, cudaMemcpyHostToDevice);
    
    minimum<<<1,n/2>>>(destination);
    
    cudaMemcpy(&result, destination, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The minimum element is "<<result<<endl;

    //MAX
     cudaMemcpy(destination, source, size, cudaMemcpyHostToDevice);
    maximum<<<1,n/2>>>(destination);
    cudaMemcpy(&result, destination, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The maximum element is "<<result<<endl;

   
    
    
    
    
    

    //SD
    cudaMemcpy(destination, source, size, cudaMemcpyHostToDevice);
    sumOfSquares<<<1,n>>>(destination);
    cudaMemcpy(&result, destination, sizeof(int), cudaMemcpyDeviceToHost);
    sum<<<1,n/2>>>(destination);
    cudaMemcpy(&result, destination, sizeof(int), cudaMemcpyDeviceToHost);
    
    float sd=(result*1.0)/n -(avg*avg);
    cout<<"The Variance  is "<<sd<<endl;
    sd=sqrt(sd);
    cout<<"The standard Deviation is "<<sd<<endl;
       
    
    
    
           
    return 0;
}

4  7  8  6  4  6  7  3  
The sum  is 45
The average  is 5.625
The minimum element is 3
The maximum element is 8
The Variance  is 2.73438
The standard Deviation is 1.65359



In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
